In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from scipy.stats import norm
from statsmodels.base.model import GenericLikelihoodModel
import seaborn as sns
sns.set_palette("muted")
sns.set_color_codes()
sns.set_style("ticks")
sns.set_style({"xtick.direction": "in","ytick.direction": "in"})
sns.set_style({"axes.grid": "True", "grid.color": "0.95"})

plt.rcParams["figure.figsize"] = [6,3]
plt.rcParams["figure.dpi"] = 100

In [ ]:
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
import root_pandas
import glob

In [ ]:
df_signal = root_pandas.read_root(glob.glob('K3pi/realistic_mc/DSRho-mdst_K3pi_basf2_00_svd*.root'), columns=['nocand'])

In [ ]:
df_all = root_pandas.read_root(glob.glob('K3pi/realistic_mc/*.root'), columns=['nocand'])

In [ ]:
plt.hist(df_all['nocand'], log=True, bins=5, align='left', range=(1,6), label='generic MC', hatch='//', edgecolor='b', fill=False, histtype='step', linewidth=2)
plt.hist(df_signal['nocand'], log=True, bins=5, align='left', range=(1,6), label='signal MC', hatch='\\\\', edgecolor='g', fill=False, histtype='step', linewidth=2)
plt.locator_params(axis='x', nbins=6)
plt.ylim(bottom=1)
plt.legend()
plt.savefig("K3pi_nocand_hist.pdf")

In [ ]:
df_data = root_pandas.read_root(glob.glob('K3pi/*.root'))

In [ ]:
plt.hist(df_all['nocand'], log=True, bins=5, align='left', range=(1,6), label='MC')
plt.hist(df_data['nocand'], log=True, bins=5, align='left', range=(1,6), label='data')
plt.xlabel("Number of candidates")
plt.legend()
plt.locator_params(axis='x', nbins=6)

In [ ]:
plt.hist(df_all['nocand'] / df_data['nocand'], bins=5, align='left', range=(1,6))
plt.xlabel("Number of candidates")
plt.locator_params(axis='x', nbins=6)